In [1]:
import torch
import torchvision
from torchvision.datasets import MNIST

D:\2_Miniconda\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\2_Miniconda\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
dataset=MNIST(root='data/',download=True)

In [3]:
len(dataset)

60000

In [4]:
test_dataset=MNIST(root='data/',train=False)
len(test_dataset)

10000

In [5]:
dataset[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x16AA2567EB0>, 5)

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
image,label=dataset[0]


In [8]:
import torchvision.transforms as transforms

In [9]:
dataset=MNIST(root='data/',train=True,transform=transforms.ToTensor())

In [10]:
img_tensor, label =dataset[0]
print(img_tensor.shape,label)

torch.Size([1, 28, 28]) 5


In [11]:
print(img_tensor[:,10:15,10:15])
print(torch.max(img_tensor),torch.min(img_tensor))

tensor([[[0.0039, 0.6039, 0.9922, 0.3529, 0.0000],
         [0.0000, 0.5451, 0.9922, 0.7451, 0.0078],
         [0.0000, 0.0431, 0.7451, 0.9922, 0.2745],
         [0.0000, 0.0000, 0.1373, 0.9451, 0.8824],
         [0.0000, 0.0000, 0.0000, 0.3176, 0.9412]]])
tensor(1.) tensor(0.)


In [12]:
import numpy as np

def split_indices(n, val_pct):
    n_val=int(val_pct*n)
    idxs=np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]